In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras

DATA_PATH = "/content/drive/My Drive/MSc_Project_Colab/mixed_1140_1_segment_128_mels.json"

def load_data(data_path):

    with open(data_path, "r") as fp:
        data = json.load(fp)

    X = np.array(data["log mel spectrogram"])
    y = np.array(data["labels"])
    return X, y

def prepare_datasets(test_size, validation_size):

    # load data
    X, y = load_data(DATA_PATH)

    # create the train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

    # create the train/validation split
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    # TF expects a 3D array -> (# time bins, # mel bands, # colours)
    X_train = X_train[..., np.newaxis] # 4D array -> (# segments, # time bins, # mel bands, # colours)
    X_validation = X_validation[..., np.newaxis]
    X_test = X_test[..., np.newaxis]

    return X_train, X_validation, X_test, y_train, y_validation, y_test

def build_model(input_shape):

    # create model
    model = keras.Sequential()

    # 1st conv layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPool2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 2nd conv layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPool2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 3rd conv layer
    model.add(keras.layers.Conv2D(32, (2, 2), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPool2D((2, 2), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # flatten output and feed to dense layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.3))

    # output layer
    model.add(keras.layers.Dense(2, activation='softmax'))

    return model

def predict(model, X, y):

    X = X[np.newaxis, ...]

    prediction = model.predict(X)

    # extract index with max value
    predicted_index = np.argmax(prediction, axis=1)
    dataLabel = ""
    myPrediction = ""
    if y == 0:
        dataLabel = "BIRD!"
    else:
        dataLabel = "NO BIRD"

    if predicted_index == 0:
        myPrediction = "BIRD?"
    else:
        myPrediction = "NO BIRD?"

    print("Predicted: {}, Expected: {}".format(myPrediction, dataLabel))


if __name__ == "__main__":
    # create training, validation and test sets
    X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.25, 0.2)

    # build the CNN
    input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])
    model = build_model(input_shape)

    # compile the network
    optimizer = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # train the CNN
    model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=100)

    # evaluate the CNN on the test set
    test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
    print("Accuracy on test set is: {}".format(test_accuracy))

    index = 0
    # make prediction on a sample
    while index < len(X_test):
        X = X_test[index]
        y = y_test[index]
        print("Sample: {}".format(index+1))
        predict(model, X, y)
        index += 1

Epoch 1/100
22/22 [==============================] - 1s 29ms/step - loss: 0.9348 - accuracy: 0.5868 - val_loss: 0.7930 - val_accuracy: 0.5965
Epoch 2/100
22/22 [==============================] - 0s 19ms/step - loss: 0.5283 - accuracy: 0.7397 - val_loss: 0.8460 - val_accuracy: 0.5673
Epoch 3/100
22/22 [==============================] - 0s 20ms/step - loss: 0.4240 - accuracy: 0.7956 - val_loss: 0.8885 - val_accuracy: 0.5673
Epoch 4/100
22/22 [==============================] - 0s 19ms/step - loss: 0.3078 - accuracy: 0.8647 - val_loss: 0.8608 - val_accuracy: 0.5848
Epoch 5/100
22/22 [==============================] - 0s 20ms/step - loss: 0.2507 - accuracy: 0.8912 - val_loss: 0.7319 - val_accuracy: 0.6199
Epoch 6/100
22/22 [==============================] - 0s 20ms/step - loss: 0.2191 - accuracy: 0.9206 - val_loss: 0.9151 - val_accuracy: 0.5965
Epoch 7/100
22/22 [==============================] - 0s 19ms/step - loss: 0.2105 - accuracy: 0.9059 - val_loss: 0.7498 - val_accuracy: 0.6433
Epoch 